------This Jupyter notebook is authored by J. Antonio Sidaoui, jas2545@columbia.edu------

# Diffusion Maps Implementation

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn import preprocessing
from numpy import genfromtxt
from numpy import linalg

In [ ]:
# Function to ensure covariance matrices remain positive definite for numerical stability

def ensure_positive_definite(matrix, min_eigenvalue=1e-6):
    """
    Ensures the matrix is positive definite by adjusting its eigenvalues.

    Parameters:
    matrix (numpy.ndarray): The input matrix to be checked and adjusted.
    min_eigenvalue (float): The minimum eigenvalue threshold.

    Returns:
    numpy.ndarray: A positive definite matrix.
    """
    # Eigenvalue decomposition
    eigenvalues, eigenvectors = np.linalg.eigh(matrix)
    # Clip eigenvalues to ensure they are not too small
    eigenvalues = np.clip(eigenvalues, min_eigenvalue, None)
    # Reconstruct the matrix
    matrix_pd = (eigenvectors @ np.diag(eigenvalues)) @ eigenvectors.T
    return matrix_pd

In [ ]:
def compute_distances(X):
    '''
    Constructs a distance matrix from data set, assumes Euclidean distance

    Inputs:
        X       a numpy array of size n x p holding the data set (n observations, p features)

    Outputs:
        D       a numpy array of size n x n containing the euclidean distances between points

    '''

    # Initialize distance matrix D
    n = np.shape(X)[0]
    D = np.empty([n, n])

    # Precompute the inverses for efficiency
    inverses = [np.linalg.pinv(np.matmul(np.transpose(X[0:i, :]), X[0:i, :])) for i in range(n)]

    for i in range(n):
      for j in range(n):
        diff = X[i, :] - X[j, :]
        D[i, j] = 0.5 * np.linalg.multi_dot([diff, (inverses[i] + inverses[j]), diff.T]) # Compute modified Mahalanbis distance between data points i and j
                                                                                         # Note that data points are in R^p and are rows of the matrix
    # return distance matrix
    return D

In [ ]:
def compute_affinity_matrix(D, kernel_type, sigma=None, k=None):
    '''
    Construct an affinity matrix from a distance matrix via gaussian kernel.

    Inputs:
        D               a numpy array of size n x n containing the distances between points
        kernel_type     a string, either "gaussian" or "adaptive".
                            If kernel_type = "gaussian", then sigma must be a positive number
                            If kernel_type = "adaptive", then k must be a positive integer
        sigma           the non-adaptive gaussian kernel parameter
        k               the adaptive kernel parameter

    Outputs:
        W       a numpy array of size n x n that is the affinity matrix

    '''
    if kernel_type=='gaussian':
        if sigma<=0:  # Check if sigma is valid
            return 'error, sigma must be a positive number' # return error if not
        else:
            W=np.exp((-(D**2))/(sigma**2)) # Compute affinity matrix using Gaussian Kernel

    if kernel_type=='adaptive':
        if type(k)!=int or k<=0: # Check if t is valid
            return 'error, k must be a positive integer' # return error if not
        else:
            n=np.shape(D)[0]
            W=np.empty([n,n]) # Initialize W
            D_sort=np.sort(D, axis=1) # Sort rows of D from smallest to largest
            for i in range(n):
                for j in range(n):
                    # the denominators here take the k'th element in the i'th or j'th row of the sorted D
                    W[i, j]=.5*(np.exp(-(D[i, j]**2)/(D_sort[i, :][k]**2))+
                                np.exp(-(D[i, j]**2)/(D_sort[j, :][k]**2)))

    # return the affinity matrix
    return W

In [ ]:
def diff_map_info(W):
    '''
    Construct the information necessary to easily construct diffusion map for any t

    Inputs:
        W           a numpy array of size n x n containing the affinities between points

    Outputs:

        diff_vec    a numpy array of size n x n-1 containing the n-1 nontrivial eigenvectors of Markov matrix as columns
        diff_eig    a numpy array of size n-1 containing the n-1 nontrivial eigenvalues of Markov matrix

        We assume the convention that the coordinates in the diffusion vectors are in descending order
        according to eigenvalues.
    '''
    # Compute the matrix of row sums D^(-1/2)
    D=np.diag((np.sum(W, axis=1))**(-.5))

    # Construct symmetric matrix M_s
    M_s=D@W@D

    # Find eigenvalues and eigenvectors of M_s
    w, v=np.linalg.eigh(M_s)

    # Sort eigenvalues and corresponding eigenvectors in descending order
    eigvals=-np.sort(-w)
    eigvecs=v[:, eigvals.argsort()]

    # Chop off the trivial eigenvector and eigenvalue
    eigvals=eigvals[1:]
    eigvecs=eigvecs[:, 1:]

    # Get the eigenpairs
    diff_eig=eigvals # eigenvalues of Markov matrix are the same as those of M_s
    n=np.shape(W)[1]
    diff_vec=np.empty([n, n-1]) # initialize diff_vec

    # Compute normalized eigenvectors of the Markov matrix
    for i in range(np.shape(eigvecs)[1]):
        diff_vec[:, i]=(D@eigvecs[:, i])/np.linalg.norm(D@eigvecs[:, i])

    # return the info for diffusion maps
    return diff_vec, diff_eig

In [ ]:
def get_diff_map(diff_vec, diff_eig, t):
    '''
    Construct a diffusion map at t from eigenvalues and eigenvectors of Markov matrix

    Inputs:
        diff_vec    a numpy array of size n x n-1 containing the n-1 nontrivial eigenvectors of Markov matrix as columns
        diff_eig    a numpy array of size n-1 containing the n-1 nontrivial eigenvalues of Markov matrix
        t           diffusion time parameter t

    Outputs:
        diff_map    a numpy array of size n x n-1, the diffusion map defined for t
    '''
    # Raise each eigenvalue to the t'th power and multiply the corresponding eigenvector by it
    diff_map=diff_vec*(diff_eig**t)

    return diff_map

In [ ]:
# Choose k dominant eigenvalues as dimension of embedding space and number of latent variables

def embedding_dimension(diff_eig):
  consecutive_distances = np.abs(np.diff(diff_eig))
  dimension=np.argmax(consecutive_distances) # find the largest spectral gap to set the dimension
  return dimension

# Conditional Sampling & JDKF

In [ ]:
def is_positive_semidefinite(matrix):
    """Checks if a matrix is positive semidefinite."""
    try:
        # Attempt Cholesky decomposition
        np.linalg.cholesky(matrix)
        return True  # If successful, it's positive semidefinite
    except np.linalg.LinAlgError:
        return False  # If Cholesky fails, it's not

In [ ]:
# Sample diffusion coordinates at t+1 conditional on the scenario and information up to time t

def conditional_sample_H(F, Q, H, psi_t, fixed_indices, fixed_values):
    """
    Sample psi_{t+1} given psi_t and a subset of coordinates of H*psi_{t+1} being fixed.

    Parameters:
    F (numpy.ndarray): State transition matrix.
    Q (numpy.ndarray): Process covariance matrix.
    H (numpy.ndarray): Lifting operator from embedding space to original space.
    psi_t (numpy.ndarray): Current state.
    fixed_indices (list of int): Indices of the coordinates of H*psi_{t+1} that are fixed (factors to be stressed).
    fixed_values (numpy.ndarray): Values to which the coordinates of H*psi_{t+1} are fixed (scenario vector).

    Returns:
    numpy.ndarray: Sampled psi_{t+1} conditioned on the fixed values.
    """
    # Predict the mean and covariance of psi_{t+1} given psi_t
    mu_t1 = F @ psi_t
    cov_t1 = Q

    # Predict the mean and covariance in the measurement space
    mu_t1_obs = H @ mu_t1
    cov_t1_obs = H @ cov_t1 @ H.T

    # Partition the mean and covariance based on fixed_indices in the measurement space
    all_indices_obs = np.arange(len(mu_t1_obs))
    free_indices_obs = np.setdiff1d(all_indices_obs, fixed_indices)

    mu_t1_obs_fixed = mu_t1_obs[fixed_indices]
    mu_t1_obs_free = mu_t1_obs[free_indices_obs]

    cov_t1_obs_ff = cov_t1_obs[np.ix_(free_indices_obs, free_indices_obs)]
    cov_t1_obs_fi = cov_t1_obs[np.ix_(free_indices_obs, fixed_indices)]
    cov_t1_obs_if = cov_t1_obs[np.ix_(fixed_indices, free_indices_obs)]
    cov_t1_obs_ii = cov_t1_obs[np.ix_(fixed_indices, fixed_indices)]

    if not is_positive_semidefinite(cov_t1_obs_ii):
        cov_t1_obs_ii = ensure_positive_definite(cov_t1_obs_ii)

    # Conditional mean and covariance in the measurement space
    mu_cond_obs = mu_t1_obs_free + cov_t1_obs_fi @ np.linalg.inv(cov_t1_obs_ii) @ (fixed_values - mu_t1_obs_fixed)
    cov_cond_obs = cov_t1_obs_ff - cov_t1_obs_fi @ np.linalg.inv(cov_t1_obs_ii) @ cov_t1_obs_if

    # Convert the conditional mean and covariance back to the latent space
    H_free = H[free_indices_obs]
    mu_cond_latent = np.linalg.pinv(H_free) @ mu_cond_obs
    cov_cond_latent = np.linalg.pinv(H_free) @ cov_cond_obs @ np.linalg.pinv(H_free).T

    if not is_positive_semidefinite(cov_cond_latent):
        cov_cond_latent = ensure_positive_definite(cov_cond_latent)


    # Sample from the conditional distribution
    psi_t1_free_sampled = np.random.multivariate_normal(mu_cond_latent, cov_cond_latent)

    # Construct the full psi_{t+1} sample
    psi_t1 = np.empty_like(mu_t1)
    psi_t1 = psi_t1_free_sampled

    return psi_t1

In [ ]:
def kalman_filter_joint(y, z, F, H_y, Q, R_y, A, R_z, R_yz, x0, P0):
  """
    Kalman filter with joint update using both y_t and z_t.

    Parameters:
    - y: Observations of y_t (n_timesteps, n_y)
    - z: Observations of z_t (n_timesteps, n_z)
    - F: State transition matrix (n_x, n_x)
    - H_y: Lifting operator matrix for y_t (n_y, n_x)
    - Q: Process noise covariance (n_x, n_x)
    - R_y: Observation noise covariance for y_t (n_y, n_y)
    - A: Matrix linking y_t to z_t (n_z, n_y)
    - R_z: Observation noise covariance for z_t (n_z, n_z)
    - R_yz: Covariance between y_t and z_t (n_y, n_z)
    - x0: Initial state estimate (n_x,)
    - P0: Initial state covariance (n_x, n_x)

    Returns:
    - x_est: Estimated states over time (n_timesteps, n_x)
    - P_est: Estimated state covariance over time (n_timesteps, n_x, n_x)
    - log_likelihood: Total log-likelihood for the observations
    """
    T = y.shape[0]
    n_x = F.shape[0]

    x_est = np.zeros((T, n_x))
    P_est = np.zeros((T, n_x, n_x))

    x_t = x0
    P_t = P0
    log_likelihood = 0.0

    H_joint = np.vstack([H_y, A @ H_y])
    R_joint = np.block([
        [R_y,    R_yz],
        [R_yz.T, R_z ]
    ])
    R_joint = ensure_positive_definite(R_joint)

    for t in range(T):
        z_joint_t = np.hstack([y[t], z[t]])

        x_pred = F @ x_t
        P_pred = F @ P_t @ F.T + Q

        innovation = z_joint_t - H_joint @ x_pred
        S = H_joint @ P_pred @ H_joint.T + R_joint
        S = ensure_positive_definite(S)

        K = P_pred @ H_joint.T @ np.linalg.pinv(S)
        x_t = x_pred + K @ innovation
        P_t = (np.eye(n_x) - K @ H_joint) @ P_pred
        P_t = ensure_positive_definite(P_t)

        x_est[t] = x_t
        P_est[t] = P_t

        ll = -0.5 * (np.log(np.linalg.det(S) + 1e-10) +
                     innovation.T @ np.linalg.pinv(S) @ innovation +
                     len(z_joint_t) * np.log(2 * np.pi))
        log_likelihood += ll

    return x_est, P_est, log_likelihood

In [ ]:
def rts_smoother(x_filt, P_filt, F, Q):
  """
    Implementation of the RTS smoother.

    Parameters:
    - x_filt: Filtered states (n_timesteps, n_x)
    - P_filt: Filtered state covariances (n_timesteps, n_x, n_x)
    - F: State transition matrix (n_x, n_x)
    - Q: Process noise covariance (n_x, n_x)
    Returns:
    - x_smooth: Smoothed states (n_timesteps, n_x)
    - P_smooth: Smoothed state covariances (n_timesteps, n_x, n_x)
    """
    T, n = x_filt.shape
    x_smooth = np.zeros_like(x_filt)
    P_smooth = np.zeros_like(P_filt)

    x_smooth[-1] = x_filt[-1]
    P_smooth[-1] = P_filt[-1]

    for t in reversed(range(T - 1)):
        P_pred = F @ P_filt[t] @ F.T + Q
        G = P_filt[t] @ F.T @ np.linalg.pinv(P_pred)

        x_smooth[t] = x_filt[t] + G @ (x_smooth[t+1] - F @ x_filt[t])
        P_smooth[t] = P_filt[t] + G @ (P_smooth[t+1] - P_pred) @ G.T

    return x_smooth, P_smooth

In [ ]:
def maximization_step(y, z, x_smooth, H_y):
    """
    Perform the maximization step of the JDKF algorithm.

    Parameters:
    - y: Observations of y_t (n_timesteps, n_y)
    - z: Observations of z_t (n_timesteps, n_z)
    - x_smooth: Smoothed states (n_timesteps, n_x)
    - H_y: Lifting operator matrix for y_t (n_y, n_x)
    Returns:
    - A: Matrix linking y_t to z_t (n_z, n_y)
    - R_z: Observation noise covariance for z_t (n_z, n_z)
    - R_y: Observation noise covariance for y_t (n_y, n_y)
    - R_yz: Covariance between y_t and z_t (n_y, n_z)
    """
    T = x_smooth.shape[0]
    Hx = (H_y @ x_smooth.T).T  # (T, n_y)

    # Estimate A: z = A (H_y x)
    A = np.linalg.lstsq(Hx, z, rcond=None)[0].T  # (n_z, n_y)

    # Residuals
    res_y = y - Hx
    res_z = z - (A @ Hx.T).T

    # Covariance estimates
    R_y  = (res_y.T @ res_y) / T
    R_z  = (res_z.T @ res_z) / T
    R_yz = (res_y.T @ res_z) / T

    return A, R_z, R_y, R_yz

In [ ]:
def em_kalman_filter(y, z, F, H_y, Q, x0, P0, A_init=None, R_y_init=None, R_z_init=None, R_yz_init=None, max_iters=100, tol=1e-6):
  """
    Implementation of the Expectation-Maximization (EM) algorithm for the JDKF.

    Parameters:
    - y: Observations of y_t (n_timesteps, n_y)
    - z: Observations of z_t (n_timesteps, n_z)
    - F: State transition matrix (n_x, n_x)
    - H_y: Lifting operator matrix for y_t (n_y, n_x)
    - Q: Process noise covariance (n_x, n_x)
    - x0: Initial state estimate (n_x,)
    - P0: Initial state covariance (n_x, n_x)
    - A_init: Initial matrix linking y_t to z_t (n_z, n_y)
    - R_y_init: Initial observation noise covariance for y_t (n_y, n_y)
    - R_z_init: Initial observation noise covariance for z_t (n_z, n_z)
    - R_yz_init: Initial covariance between y_t and z_t (n_
    Returns:
    - A: Matrix linking y_t to z_t (n_z, n_
    - R_z: Observation noise covariance for z_t (n_z, n_z)
    - R_y: Observation noise covariance for y_t (n_y, n_y)
    - R_yz: Covariance between y_t and z_t (n_y, n_z)
    - x_smooth: Smoothed states (n_timesteps, n_x)
    - x_filt: Filtered states (n_timesteps, n_x)
    - log_likelihoods: Log-likelihoods at each iteration
    """
    T, n_y = y.shape
    n_z = z.shape[1]

    A = np.random.randn(n_z, n_y) if A_init is None else A_init
    R_y = np.eye(n_y) * 0.1 if R_y_init is None else R_y_init
    R_z = np.eye(n_z) * 0.1 if R_z_init is None else R_z_init
    R_yz = np.zeros((n_y, n_z)) if R_yz_init is None else R_yz_init

    log_likelihoods = []

    for it in range(max_iters):
        x_filt, P_filt, log_lik = kalman_filter_joint(y, z, F, H_y, Q, R_y, A, R_z, R_yz, x0, P0)
        x_smooth, P_smooth = rts_smoother(x_filt, P_filt, F, Q)

        log_likelihoods.append(log_lik)
        print(f"Iter {it}, log-likelihood: {log_lik:.4f}")

        # Use relative change for convergence
        if it > 0:
            delta_ll = abs(log_likelihoods[-1] - log_likelihoods[-2])
            rel_change = delta_ll / max(abs(log_likelihoods[-2]), 1e-10)
            print(f"Relative log-likelihood change: {rel_change}")
            if rel_change < tol:  # threshold
                print(f"Converged by relative log-likelihood change at iteration {it}")
                break

        A, R_z, R_y, R_yz = maximization_step(y, z, x_smooth, H_y)

    return A, R_z, R_y, R_yz, x_smooth, x_filt, log_likelihoods